In [80]:
import cartopy
import fiona as fio
import matplotlib.pyplot as plt
import rasterio
import rasterio.crs
import rasterio.warp
import shapely.geometry

## Reference

1. http://stackoverflow.com/questions/13856123/setting-up-a-map-which-crosses-the-dateline-in-cartopy
2. http://gis.stackexchange.com/questions/143294/plot-points-on-map-using-fiona-and-or-shapely

## Setup

### Setup NLCD projection

In [81]:
nlcd_albers = cartopy.crs.AlbersEqualArea(central_longitude=-96, central_latitude=23,
                                          standard_parallels=(29.5, 45.5))

## `shapely`

In [83]:
tile_lon, tile_lat = -72.0, 41.0

In [84]:
bbox = [(tile_lon, tile_lat), (tile_lon + 1, tile_lat), (tile_lon + 1, tile_lat + 1), (tile_lon, tile_lat + 1)]

In [85]:
tile = shapely.geometry.Polygon(bbox)
tile_proj = cartopy.crs.PlateCarree()

## Plot with `cartopy` and `shapely`

In [77]:
# Setup axes
ax = plt.axes(projection=nlcd_albers)
# Add default features -- coastlines & stock image
ax.coastlines()
# ax.stock_img()

ax.add_geometries([tile], tile_proj, facecolor='coral', edgecolor='k', alpha=0.5)
ax.add_geometries([nlcd_tile], nlcd_albers, facecolor='blue', edgecolor='k', alpha=0.5)
plt.show()

In [12]:
# NLCD 1992: http://webmap.ornl.gov/ogcdown/wcsdown.jsp?dg_id=10009_21
# Projection: USA Contiguous Albers Equal Area Conic (NLCD)
# Spatial Extent: N: 3177735, S: 267885, E: 2266005, W: -2361915
nlcd_n, nlcd_s, nlcd_e, nlcd_w = 3177735, 267885, 2266005, -2361915
nlcd_tile = shapely.geometry.Polygon([(nlcd_e, nlcd_n),
                                      (nlcd_w, nlcd_n),
                                      (nlcd_w, nlcd_s),
                                      (nlcd_e, nlcd_s)])

## Tile coordinates

In [13]:
tile_albers_x, tile_albers_y = nlcd_albers.transform_point(tile_lon, tile_lat, tile_proj)
tile_albers_x, tile_albers_y

(1981435.104505844, 2250528.3325550687)

In [59]:
nlcd_tile.bounds

(-2361915.0, 267885.0, 2266005.0, 3177735.0)

In [72]:
nlcd_w, tile_albers_x

(-2361915, 1981435.104505844)

In [103]:
def match_to_grid(input_xy, grid_xy, pix_size):
    """ Return new postings for input that coalign with grid
    
    Args:
      input_ul (tuple): X/Y coordinate of input extent to be repositioned
      grid_ul (tuple): X/Y coordinate of grid extent to match
      pix_size (tuple): X/Y pixel sizes of data to match
      
    Returns:
      tuple: new X/Y coordinate of matched input 

    """
    new_xy = []
    for g_xy, i_xy, ps in zip(grid_xy, input_xy, pix_size):
        offset = int(round((g_xy - i_xy) / ps))
        new_xy.append(g_xy - offset * ps)
    return new_xy

In [105]:
match_to_grid((tile_albers_x, tile_albers_y),
              (nlcd_w, nlcd_n),
              (30, 30))

[1981425, 2250555]

## `rasterio`

In [69]:
nlcd_crs_proj4 = '+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=23 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs '
nlcd_crs = rasterio.crs.from_string(nlcd_crs_proj4)

In [86]:
tile_lon, tile_lat = -72.0, 41.0

rasterio.warp.transform({'init': 'epsg:4326'}, 
                        nlcd_crs,
                        [tile_lon], 
                        [tile_lat])

([1981435.1045201581], [2250528.3325191094])

In [94]:
tile_bounds = (tile_lon, tile_lat, tile_lon + 1, tile_lat + 1)

bounds_albers =  rasterio.warp.transform_bounds(
    {'init': 'epsg:4326'},
    nlcd_crs,
    tile_lon, tile_lat, tile_lon + 1, tile_lat + 1)

In [95]:
bounds_albers

(1953469.5791992692, 2250528.3325191094, 2062121.0342337284, 2379874.020125433)

In [111]:
print(match_to_grid(bounds_albers[0:2], (nlcd_w, nlcd_n), (30, 30)))
print(match_to_grid(bounds_albers[2:4], (nlcd_w, nlcd_n), (30, 30)))

[1953465, 2250525]
[2062125, 2379885]
